Por si solos, los modelos de lenguaje no pueden realizar acciones, solo producir texto de salida. El uso de LangChain y LangGraph es la creación de agentes. Los agentes son sistemas que usan LLMs como motores de razonamiento que determinan que tareas realizar y que entradas son necesarias para completar la acción.

Despues de que la acción sea completada, los resultados pueden ser usados para alimentar el contexto del LLM para determinar si más acciones son necesarias o ya esta completado todo lo requerido. Esto normalmente se logra con tool-calling.

# Define tools

Primero es necesario crear las herramientas que deseamos usar. Aquí se presenta el uso de Tavily, un motor de busqueda.

LangChain provee una herramienta para usar Tavily como una tool.

In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("What is the weather in SF")

print(search_results)


tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1741547954, 'localtime': '2025-03-09 12:19'}, 'current': {'last_updated_epoch': 1741547700, 'last_updated': '2025-03-09 12:15', 'temp_c': 13.9, 'temp_f': 57.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 4.9, 'wind_kph': 7.9, 'wind_degree': 220, 'wind_dir': 'SW', 'pressure_mb': 1023.0, 'pressure_in': 30.2, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 49, 'cloud': 50, 'feelslike_c': 13.6, 'feelslike_f': 56.5, 'windchill_c': 10.1, 'windchill_f': 50.1, 'heatindex_c': 10.7, 'heatindex_f': 51.2, 'dewpoint_c': 8.4, 'dewpoint_f': 47.2, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 3.9, 'gust_mph': 6.1, 'gust_kph': 9.9}}"}, {'url': 'https://world-weather.info/forecast/usa

# Using LLMs

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-3.5-turbo", model_provider="openai")

Para permitir al modelo realizar tool caling, debemos usar .bind_tools para dar conocimiento al modelo de las herramientas:

In [3]:
model_with_tools = model.bind_tools(tools)

Ahora llamaremos al modelo con la tool. Primero se llama al modelo con un mensaje normal; se pueden leer los campos content y tool_calls del response para ver como el modelo con la tool reacciona.

In [4]:
from langchain_core.messages import HumanMessage, AIMessage

response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


Ahora haremos una prompt donde sí esperariamos que el modelo responda haciendo uso de la tool:

In [5]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in CDMX?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in CDMX'}, 'id': 'call_djFcofJxFhj6u4ZHWBWFthwl', 'type': 'tool_call'}]


Podemos ver que no hay texto alguno en el contenido pero sí hubo una llamada a la tool. El modelo quiere hacer uso de la tool.

Este codigo no esta haciendo uso de la herramienta aún, solo nos esta diciendo que quiere hacer uso de esta. Para poder usarla/llamarla, debemos definir el agente.

# Create the agent

Ahora que ya hemos declarado el modelo y la tool, ya podemos crear nuestro agente. Haremos uso de LangGraph para construir el agente. Se hara uso de una interfaz de alto nivel para construir el agente, las ventajas de LangGraph es que esta intefaz de alto nive esta hecha sobre bases de bajo nivel, bastante controlables con su API, y que permiten modificaciones en caso de que sea requerido.

Ahora podemos inicializar el agente con el LLM y sus tools.

Podrás notar que estamos pasando el model, no model_with_tools. Esto es porque create_react_agent llamará a .bind_tools por debajo.

In [6]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

# Run the agent

Ahora se puede ejecutar el agente, pero hay que tomar en cuenta que son stateless, por lo que no preservan el historial de conversaciones. Nota que el agente retorna el estado final al final de la interacción.

In [7]:
response = agent_executor.invoke({"messages":[HumanMessage(content="hi")]})

response["messages"]

[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='4308a118-b4ef-459d-a9c8-b78cb66888f3'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 82, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6fa9d972-56f1-4214-a433-61a60b5c0426-0', usage_metadata={'input_tokens': 82, 'output_tokens': 11, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

Ahora observemos un ejemplo donde se haga el llamado a la tool.

In [8]:
response = agent_executor.invoke(
    {"messages":[HumanMessage(content="What's the weather in CDMX?")]}
)

response["messages"]

[HumanMessage(content="What's the weather in CDMX?", additional_kwargs={}, response_metadata={}, id='9e1b5654-da40-4fe9-aa67-d7c747be792a'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kiQowCtZAxEriViOEVdu2hxf', 'function': {'arguments': '{"query":"weather in CDMX"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 90, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-263514c2-6c59-4e2f-a2ef-f15f202a218c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in CDMX'}, 'id': 'call_kiQowCtZAxEriViOEVdu2hxf', 'type': 'tool_call'}], usag

# Streaming messages

Por defecto, el modelo retorna toda la respuesta una vez esta sea inferida. Usando tecnicas de streamming se puede lograr devolver las partes del output final mientras este se va procesando.

Un metodo es stremeando los mensajes conforme se vayan produciendo:

In [9]:
for step in agent_executor.stream(
    {"messages":[HumanMessage(content="What's the weather in Boston?")]},
     stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather in Boston?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_5BG05baFpKQ2sTdOyVmyzvTt)
 Call ID: call_5BG05baFpKQ2sTdOyVmyzvTt
  Args:
    query: weather in Boston
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Boston', 'region': 'Massachusetts', 'country': 'United States of America', 'lat': 42.3583, 'lon': -71.0603, 'tz_id': 'America/New_York', 'localtime_epoch': 1741546953, 'localtime': '2025-03-09 15:02'}, 'current': {'last_updated_epoch': 1741546800, 'last_updated': '2025-03-09 15:00', 'temp_c': 6.7, 'temp_f': 44.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 13.0, 'wind_kph

# Streaming tokens

Alternativo a stremmear mensajes, se pueden stremear tokens. Esto se puede lograr especificando stream_mode="messages".

In [10]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| San| Francisco| is| as| follows|:
|-| Temperature|:| |55|.|0|°F| (|12|.|8|°C|)
|-| Condition|:| Part|ly| cloudy|
|-| Wind|:| |4|.|9| mph|,| coming| from| the| southwest|
|-| Pressure|:| |30|.|2| in|
|-| Hum|idity|:| |64|%
|-| Cloud| Cover|:| |50|%
|-| Fe|els| like|:| |54|.|2|°F|
|-| Visibility|:| |9|.|0| miles|

|For| more| detailed| weather| forecast| information|,| you| can| visit| [|Weather|Api|.com|](|https|://|www|.weather|api|.com|/)| or| [|World|-|Weather|.info|](|https|://|world|-|weather|.info|/|forecast|/|usa|/s|an|_fr|anc|isco|/m|arch|-|202|5|/|).|

# Adding Memory

El agente es stateless por lo que no tiene memoria de las conversaciones. Para darle memoria debemos proveerle un checkpointer junto con thread_id al hacer .invoke del modelo, para ir guardado la conversación y que el modelo tenga memoria.

In [11]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [12]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable":{"thread_id":"abc123"}}

In [13]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 85, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dc936e26-3884-4bf1-89f2-feddcd804734-0', usage_metadata={'input_tokens': 85, 'output_tokens': 12, 'total_tokens': 97, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How may I assist you further, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 108, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4564bf5e-3f06-4270-85ea-c4482c4efc97-0', usage_metadata={'input_tokens': 108, 'output_tokens': 16, 'total_tokens': 124, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----
